In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

import pickle
# Metricas
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import jaccard_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, jaccard_score, accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, jaccard_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, jaccard_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE 
import pickle

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, jaccard_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

[nltk_data] Downloading package stopwords to C:\Users\Usuario
[nltk_data]     ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Usuario
[nltk_data]     ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
df = pd.read_csv('processed_emails.csv')


In [23]:
df.head(5)

,text,spam,email_length,text_prepr
0,Subject: naturally irresistible your corporate...,1,1484,subject naturally irresistible corporate ident...
1,Subject: the stock trading gunslinger fanny i...,1,598,subject stock trading gunslinger fanny merrill...
2,Subject: unbelievable new homes made easy im ...,1,448,subject unbelievable new homes made easy im wa...
3,Subject: 4 color printing special request add...,1,500,subject color printing special request additio...
4,"Subject: do not have money , get software cds ...",1,235,subject money get software cds software compat...


In [24]:
df['text_prepr'].fillna('', inplace=True)

In [25]:
df.shape

(11300, 4)

# MÉTODOS Y SMOTE

In [13]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text_prepr'])
y = df['spam']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42, stratify = y)

print(f"Train Shape: {X_train.shape}")
print(f"Test Shape: {X_test.shape}")

Train Shape: (9040, 36489)
Test Shape: (2260, 36489)


In [57]:
modelo_vectorizer = "vectorizer.pkl"
with open (modelo_vectorizer, "wb") as file:
    pickle.dump(vectorizer, file)

In [26]:
def modelos(X_train, y_train, X_test, y_test):
    modelos = [LogisticRegression(), RandomForestClassifier(),
               AdaBoostClassifier(), GradientBoostingClassifier(), SVC(kernel = "linear"), DecisionTreeClassifier()]

    data = list()

    for model in modelos:

        print(str(model))

        model.fit(X_train, y_train)
        yhat = model.predict(X_test)

        jac = jaccard_score(y_test, yhat, average = "macro")
        acc = accuracy_score(y_test, yhat)
        pre = precision_score(y_test, yhat, average = "macro")
        rec = recall_score(y_test, yhat, average = "macro")
        f1_ = f1_score(y_test, yhat, average = "macro")
        roc = roc_auc_score(y_test, yhat)
        con = confusion_matrix(y_test, yhat)

        data.append([str(model), jac, acc, pre, rec, f1_, roc, con, model])

    df_metricas = pd.DataFrame(data = data,
                               columns = ["name", "jaccard", "accuracy", "precision", "recall", "f1", "roc_auc", "cm", "model"])
    return df_metricas

In [2]:
import pickle
with open ("model.pkl", "rb") as file:
    modelo = pickle.load(file)
    
with open("model.pkl", "wb") as file:
    pickle.dump(modelo, file, -1)

In [28]:
from imblearn.over_sampling import SMOTE 

oversampling = SMOTE(sampling_strategy=0.30)
X_balanceado, y_balanceado = oversampling.fit_resample(X, y) # Se obtienen nuevos X e y balanceados

X_train, X_test, y_train, y_test = train_test_split(X_balanceado, y_balanceado, test_size = 0.3, random_state = 0)

df_metricas = modelos(X_train, y_train, X_test, y_test)


LogisticRegression()
RandomForestClassifier()
AdaBoostClassifier()
GradientBoostingClassifier()
SVC(kernel='linear')
DecisionTreeClassifier()


In [29]:
df_metricas.sort_values(by='roc_auc', ascending=False)

,name,jaccard,accuracy,precision,recall,f1,roc_auc,cm,model
4,SVC(kernel='linear'),0.936242,0.977945,0.976964,0.957289,0.966709,0.957289,"[[2792, 18], [61, 711]]",SVC(kernel='linear')
1,RandomForestClassifier(),0.931472,0.976549,0.982182,0.948414,0.964089,0.948414,"[[2804, 6], [78, 694]]","(DecisionTreeClassifier(max_features='auto', r..."
5,DecisionTreeClassifier(),0.885084,0.957845,0.936451,0.939311,0.937871,0.939311,"[[2731, 79], [72, 700]]",DecisionTreeClassifier()
2,AdaBoostClassifier(),0.869690,0.953099,0.941784,0.917027,0.928678,0.917027,"[[2755, 55], [113, 659]]","(DecisionTreeClassifier(max_depth=1, random_st..."
0,LogisticRegression(),0.886407,0.960637,0.967086,0.915725,0.938501,0.915725,"[[2795, 15], [126, 646]]",LogisticRegression()
3,GradientBoostingClassifier(),0.866889,0.953099,0.953236,0.905753,0.926916,0.905753,"[[2779, 31], [137, 635]]",([DecisionTreeRegressor(criterion='friedman_ms...


In [30]:
print(f"Train Shape: {X_train.shape}")
print(f"Test Shape: {X_test.shape}")

Train Shape: (8358, 36489)
Test Shape: (3582, 36489)


# Random Forest

# GridSearch

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [ None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

clf = RandomForestClassifier()
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Mejores parámetros encontrados: ", grid_search.best_params_)
print("Mejor puntuación en la validación cruzada: ", grid_search.best_score_)


Mejores parámetros encontrados:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 500}
Mejor puntuación en la validación cruzada:  0.9697295405152346


In [36]:
def random_forest(df, target, leaf=None, split=None, max_depth=None, n_estimators=None):

    model = RandomForestClassifier(min_samples_leaf=leaf, 
                                   min_samples_split=split, 
                                   max_depth=max_depth, 
                                   n_estimators=n_estimators)
        
   # X_train = X_train.toarray()
    #X_test = X_test.toarray()
    model.fit(X_train, y_train)
    
    yhat = model.predict(X_test)
    model.fit(X_train, y_train)
    yhat = model.predict(X_test)
    filename = 'model.pkl'
    with open("model.pkl", "wb") as file:
        pickle.dump(model, file, -1)
    print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
    print("Accuracy:"     , accuracy_score(y_test, yhat))
    print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
    print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
    print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
    print("ROC AUC:"      , roc_auc_score(y_test, yhat)) 
    print("Accuracy:", accuracy_score(y_test, yhat))
    print("Classification Report:", classification_report(y_test, yhat))
    


In [37]:
random_forest(df, "spam", leaf = 1 , split = 10, max_depth = None, n_estimators = 500)

Jaccard Index: 0.9418585935573539
Accuracy: 0.980178671133445
Precisión: 0.9860522723079761
Sensibilidad: 0.9554247413936164
F1-score: 0.969752792391636
ROC AUC: 0.9554247413936164
Accuracy: 0.980178671133445
Classification Report:               precision    recall  f1-score   support

           0       0.98      1.00      0.99      2810
           1       1.00      0.91      0.95       772

    accuracy                           0.98      3582
   macro avg       0.99      0.96      0.97      3582
weighted avg       0.98      0.98      0.98      3582

